### Data Wrangling: Joining, Combining and Reshaping Data 

* This chapter focuces on the tools you need to combine, join and rearrange data 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import Image


In [3]:
np.random.seed(12345)
pd.options.display.max_rows = 20
plt.rc('figure', figsize=(10, 6))

### Hierarchical Indexing
* We briefly discussed hierarchical indexing in the context of `groupby`
  * 
- Hierarchical indexing allows you to have multiple index levels on an axis
- Despite the name (indexing), it applies to both indexes and columns

In [14]:
prog_languages = pd.DataFrame({ "Course":["Python", "Rust", "Python", "Rust"], 
              "Nb_participants": [10, 30, 27, 18]})
prog_languages.groupby("Course").sum()
              

,Nb_participants
Course,
Python,37
Rust,48


In [33]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    1.327195
   2   -0.919262
   3   -1.549106
b  1    0.022185
   3    0.758363
c  1   -0.660524
   2    0.862580
d  2   -0.010032
   3    0.050009
dtype: float64

In [34]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [38]:
data[('b', 1)]


0.0221845986581725

In [39]:
data['b', 1]

0.0221845986581725

In [40]:
data['b':'c']


b  1    0.022185
   3    0.758363
c  1   -0.660524
   2    0.862580
dtype: float64

In [41]:
data.loc[['b', 'd']]

b  1    0.022185
   3    0.758363
d  2   -0.010032
   3    0.050009
dtype: float64

In [46]:
data.loc[: , 2]

a   -0.919262
c    0.862580
d   -0.010032
dtype: float64

### Stacked vs. Unstacked Data

* Hierarchical indexes represent how the data is often collected
 * Ex. if variables in a hospital are measured and recorded in different files

```python
    File 1
    patient_ABC LDL 112
    patient_ABC HDL 48
    patient_CCX  LDL 112
    patient_VDM  LDL 112
    patient_ABC  VO2 112
    patient_CCZ  RER 48
    ...
```

* This format is preferred  when not all the variables are measured in all the patients

  * I.e., using columns for the variables would result in a large number of empty cells
  
* You can convert back and forth between a data frame and a hierarchical index with `stack()` and `unstack()`


In [12]:
x = data.unstack()
x

,1,2,3
a,-0.204708,0.478943,-0.519439
b,-0.555730,NaN,1.965781
c,1.393406,0.092908,NaN
d,NaN,0.281746,0.769023


In [13]:
x.stack()

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

### Hierarchical indexes on rows and columns


* In a `DataFrame`, both axes can have a hierarchical index.

In [47]:
data = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
data

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [48]:
data.index.names = ['key1', 'key2']
data.columns.names = ['state', 'color']
data

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [59]:
print(data.index.get_level_values("key1"))
print("*" * 55)
print(data.index.get_level_values("key2"))

Index(['a', 'a', 'b', 'b'], dtype='object', name='key1')
*******************************************************
Int64Index([1, 2, 1, 2], dtype='int64', name='key2')


- What type of variable would the following return (DataFrame, Series, np.array,...)?

```python
    data['Ohio']`
```


- How about the following?

```python
    frame['Ohio', 'Geeen']`
```


In [65]:
print(type(data['Ohio']))
print("*" * 40)
data['Ohio']

<class 'pandas.core.frame.DataFrame'>
****************************************


color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [66]:
print(type(data['Ohio', 'Green']))
data['Ohio', 'Green']

<class 'pandas.core.series.Series'>


key1  key2
a     1       0
      2       3
b     1       6
      2       9
Name: (Ohio, Green), dtype: int64

- You can create a `MultiIndex` and assign it manually if needed


In [67]:
data = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]])
data.index.names = ["letter", "number"]
data

0   1   2
letter number           
a      1       0   1   2
       2       3   4   5
b      1       6   7   8
       2       9  10  11

In [26]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                       names=['state', 'color'])

MultiIndex(levels=[['Colorado', 'Ohio'], ['Green', 'Red']],
           labels=[[1, 1, 0], [0, 1, 0]],
           names=['state', 'color'])

In [69]:
data.columns = pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                       names=['state', 'color'])
data

state          Ohio     Colorado
color         Green Red    Green
letter number                   
a      1          0   1        2
       2          3   4        5
b      1          6   7        8
       2          9  10       11

### Reordering and Sorting Levels
- Swap levels takes two levels `numbers` or `names` and returns a new object with the levels interchanged

In [70]:
data.index.names = ['key1', 'key2']
data

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [71]:
data.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

- You can also pass the level's index, instead of name

In [73]:
data.sort_index(level=0, ascending=False)


state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
b    2        9  10       11
     1        6   7        8
a    2        3   4        5
     1        0   1        2

In [75]:
data.swaplevel(0, 1).sort_index(level=0, ascending=False)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
2    b        9  10       11
     a        3   4        5
1    b        6   7        8
     a        0   1        2

In [77]:
data

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [76]:
data.swaplevel('state', 'color', axis=1)

color     Green  Red    Green
state      Ohio Ohio Colorado
key1 key2                    
a    1        0    1        2
     2        3    4        5
b    1        6    7        8
     2        9   10       11

### Summary Statistics by Level

- The summary statistics that we've seen before can also be applied by level
- When you apply a function on the level, the remaining levels get `"squashed"`


In [35]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

### Setting and Resetting and Index
* It' common to read in a data frame file and use one of its columns as an index
  * Setting a column as an index is done using `set_index()`
    * By default, the columns are removed from the `DataFrame`, but you can choose to leave them in 

  * `reset_index()` does the opposite; moves the index as a new column of the `DataFrame`
    * Creates a new index from an `IndexRange()`
      

In [78]:
data = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
data

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [80]:
data2 = data.set_index('b')
data2

,a,c,d
b,,,
7,0,one,0
6,1,one,1
5,2,one,2
4,3,two,0
3,4,two,1
2,5,two,2
1,6,two,3


In [82]:
data3 = data.set_index(['c', 'd'])
data3

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [83]:
data.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [84]:
data4 = data2.reset_index()
data4

,b,a,c,d
0,7,0,one,0
1,6,1,one,1
2,5,2,one,2
3,4,3,two,0
4,3,4,two,1
5,2,5,two,2
6,1,6,two,3


In [44]:
frame4.index

RangeIndex(start=0, stop=7, step=1)

### Combining and merging datasets

* You can combine `DataFrame`s using different strategies
 * You can achieve similar effect as `join` in `SQL`
 * You can also concatenate DataFrame horizontally or vertically
 
* The operation we will cover and use are

- `merge()`: similar `SQL Join`
- `concat()`: concatenate data (horizontally) or stacks it (vertically) 


In [85]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)}, columns=["key", "data1"])
df1


,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [86]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)}, columns=["key", "data2"])

df2

,key,data2
0,a,0
1,b,1
2,d,2


### Database-Style DataFrame Joins

* Merge on a common `keys`

* We can merge `df1` and `df2` on the common column name `key`  
  * `pandas` can infer the overlapping `key`, or you can specify it explicitly using `on=` 
  * Using `on=` avoids mistakes in large dataset with dozens of columns
* Since some values are duplicated in `df1`, this amount to a `many-to-one` join


In [90]:
merged_df = pd.merge(df1, df2)
merged_df.sort_values("key")

,key,data1,data2
3,a,2,0
4,a,4,0
5,a,5,0
0,b,0,1
1,b,1,1
2,b,6,1


In [91]:
merged_df = pd.merge(df1, df2, on='key')
merged_df.sort_values("key")

,key,data1,data2
3,a,2,0
4,a,4,0
5,a,5,0
0,b,0,1
1,b,1,1
2,b,6,1


- If the columns are different in both datasets, you can specify them separately using `left_on=` and `right_on=`

In [96]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})
merged_df = pd.merge(df3, df4, left_on='lkey', right_on='rkey')
merged_df[["lkey", "rkey", "data1", "data2"]]

,lkey,rkey,data1,data2
0,b,b,0,1
1,b,b,1,1
2,b,b,6,1
3,a,a,2,0
4,a,a,4,0
5,a,a,5,0


### Unmatches values

![](https://www.dropbox.com/s/u0wuwrpk8vga60h/missing_keys.png?dl=1)

* What happened to lkey=`c`   and rkey=`d`? Why aren't they showing in the resulting `DataFrame`
  * Unmatched keys are dropped from the merged result



### Inner joins

- By default, the merge does an `inner join`

#### Different join types in `pandas.join()`

| Option        | Behavior           |
| ------------- |:------------- | 
| `inner`      | Use set of Keys present in both tables (intersection) | 
| `left`      | Use keys found in left table      | 
| `right` | Use keys found in right table      | 
| `outer` | Use the union of keys found in right and left tables      | 


In [48]:
pd.merge(df1, df2,on='key', how='outer')

,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


In [50]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [51]:
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})
df2


,data2,key
0,0,a
1,1,b
2,2,a
3,3,b
4,4,d


In [52]:
pd.merge(df1, df2, on='key', how='left')

,data1,key,data2
0,0,b,1.0
1,0,b,3.0
2,1,b,1.0
3,1,b,3.0
4,2,a,0.0
5,2,a,2.0
6,3,c,NaN
7,4,a,0.0
8,4,a,2.0
9,5,b,1.0


In [53]:
pd.merge(df1, df2, how='inner')

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,5,b,1
5,5,b,3
6,2,a,0
7,2,a,2
8,4,a,0
9,4,a,2


In [54]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [55]:
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [131]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


### Avoid column collision

![](https://www.dropbox.com/s/hsmckm5l9hc77os/tables.png?dl=1)

* `pandas` renames similar columns to avoid ambiguity
* When `join()` creates ambiguity, `pandas` renames identical columns across tables Adds suffix '_x' to the left table and `_y` to the right table 

In [56]:
pd.merge(left, right, on='key1')


,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [57]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### Merging on Index

* Merging on the index is also very common
* Merge on one or more `DataFrames` with hierarchical keys operates the same way as a merge on multiple columns

In [59]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [60]:
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
right1


,group_val
a,3.5
b,7.0


In [61]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [62]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [69]:
left_hierarchial = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                               'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})
left_hierarchial


,data,key1,key2
0,0.0,Ohio,2000
1,1.0,Ohio,2001
2,2.0,Ohio,2002
3,3.0,Nevada,2001
4,4.0,Nevada,2002


In [70]:
right_hierarchial = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                              'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])

right_hierarchial

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [71]:
pd.merge(left_hierarchial, right_hierarchial, left_on=['key1', 'key2'], right_index=True)


,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4,5
0,0.0,Ohio,2000,6,7
1,1.0,Ohio,2001,8,9
2,2.0,Ohio,2002,10,11
3,3.0,Nevada,2001,0,1


In [66]:
pd.merge(left_hierarchial, right_hierarchial, left_on=['key1', 'key2'],
         right_index=True, how='outer')

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4.0,5.0
0,0.0,Ohio,2000,6.0,7.0
1,1.0,Ohio,2001,8.0,9.0
2,2.0,Ohio,2002,10.0,11.0
3,3.0,Nevada,2001,0.0,1.0
4,4.0,Nevada,2002,NaN,NaN
4,NaN,Nevada,2000,2.0,3.0


In [67]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])

left2


,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [68]:
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])
right2


,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [157]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


### Concatenating Along an Axis

- Also known as binding or stacking
- By default, concatenation 
  - Operates on `axis=0`
  - Uses `join=outer`


In [72]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [73]:
np.concatenate([arr, arr])

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [74]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [100]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

print(s1)

print("*" * 10)
print(s2)

print("*" * 10)
print(s3)

a    0
b    1
dtype: int64
**********
c    2
d    3
e    4
dtype: int64
**********
f    5
g    6
dtype: int64


In [101]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [102]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [103]:
print(s1)
print("*" * 10)
print(s3)
print("*" * 10)

s4 = pd.concat([s1, s3])
s4


a    0
b    1
dtype: int64
**********
f    5
g    6
dtype: int64
**********


a    0
b    1
f    5
g    6
dtype: int64

In [104]:
print(pd.concat([s1, s4], axis=1))
print("*" * 10 )
print(pd.concat([s1, s4], axis=1, join='inner'))


     0  1
a  0.0  0
b  1.0  1
f  NaN  5
g  NaN  6
**********
   0  1
a  0  0
b  1  1


### Creating indexes by concatenation

* Concatenation is useful for creating a hierarchical index of the data. 
  * use `keys=` pamareter of the same size as the array objects to concatenate
  * If `axis=1`, `keys` are the column (header) names when concatenating `Series`
  * If `axis=1`, `keys` are the level names when concatenating `DataFrames`
  

In [112]:
print(s1)
print("*" * 10)
print(s2)
print("*" * 10)
print(s3)


result = pd.concat([s1, s2, s3], keys=['patient_1', 'patient_2', 'patient_3'])
result


a    0
b    1
dtype: int64
**********
c    2
d    3
e    4
dtype: int64
**********
f    5
g    6
dtype: int64


patient_1  a    0
           b    1
patient_2  c    2
           d    3
           e    4
patient_3  f    5
           g    6
dtype: int64

In [113]:
result.unstack()

,a,b,c,d,e,f,g
patient_1,0.0,1.0,NaN,NaN,NaN,NaN,NaN
patient_2,NaN,NaN,2.0,3.0,4.0,NaN,NaN
patient_3,NaN,NaN,NaN,NaN,NaN,5.0,6.0


In [114]:
pd.concat([s1, s2, s3], axis=1, keys=['COL_1', 'COL_2', 'COL_3'])

,COL_1,COL_2,COL_3
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [115]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), 
                   index=['a', 'b', 'c'],
                   columns=['one', 'two'])

df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), 
                   index=['a', 'c'],
                   columns=['three', 'four'])
print(df1)
print("*" * 10)
print(df2)


   one  two
a    0    1
b    2    3
c    4    5
**********
   three  four
a      5     6
c      7     8


In [116]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [85]:
pd.concat({'level1': df1, 'level2': df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [86]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
print(df1)
print("*" * 10)
print(df2)

          a         b         c         d
0  1.246435  1.007189 -1.296221  0.274992
1  0.228913  1.352917  0.886429 -2.001637
2 -0.371843  1.669025 -0.438570 -0.539741
**********
          b         d         a
0  0.476985  3.248944 -1.021228
1 -0.577087  0.124121  0.302614


In [87]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,1.246435,1.007189,-1.296221,0.274992
1,0.228913,1.352917,0.886429,-2.001637
2,-0.371843,1.669025,-0.438570,-0.539741
3,-1.021228,0.476985,NaN,3.248944
4,0.302614,-0.577087,NaN,0.124121


- See the `Table 8.3` of Python for Data Analysis for other useful `concat()` arguments 


### Combining Data with Overlap

- `combine_first` A useful function for combining data using a method that is similar to numpy's where
  - Test some condition, if true take first value, else take second value
 


In [89]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan
print(a)
print("*" * 10)
print(b)


f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64
**********
f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64


In [90]:
np.where(pd.isnull(a), b, a)

array([ 0. ,  2.5,  2. ,  3.5,  4.5,  nan])

In [91]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})
print(df1)
print("*" * 20)
print(df2)
print("*" * 20)
df1.combine_first(df2)

     a    b   c
0  1.0  NaN   2
1  NaN  2.0   6
2  5.0  NaN  10
3  NaN  6.0  14
********************
     a    b
0  5.0  NaN
1  4.0  3.0
2  NaN  4.0
3  3.0  6.0
4  7.0  8.0
********************


,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN
